# Strojový překlad pomocí rekurentních neuronových sítí

Vyzkoušíme si strojový překlad mezi češtinou a angličtinou pomocí rekurentních neuronových sítí. K tomu použijeme model encoder-decoder, což jsou vlastně 2 RNN, které už jsme probírali a známe. Encoder čte vstupní sekvenci a vydá jeden vektor decoderu, který z něj vytvoří vstupní sekvenci. To funguje tak, že na vstupu máme větu, kterou rozsekáme na slova a každé slovo předáme encoderu, který mu dá embedding podle kontextu ostatních slov ve větě, kde si každé slovo bere informaci od předchozího a toho, co ten předchozí až do té doby zjistil. Dekodér je další RNN, která vezme výstupní vektor encoderu a vytvoří sekvenci slov, která je překladem původního vstupu a to tak, že se kouká doleva na to, co už bylo přeloženo a navíc si bere kontextový vektor z encoderu.

K natrénování si tohoto modelu použijeme [Paralelní korpus Europarl](https://www.statmt.org/europarl/), který byl získán z jednání Evropského parlamentu a obsahuje dvojjazyčné zápisy z jednání v angličtině a dalších evropských jazycích. Mezi nimi je i čestina, kterou požijeme pro náš překladač. Jak už je zřejmé z povahy dat, bude náš překladač schopný přeložit jen texty, které budou mít stejnou povahu jako zdrojová data, nebude to tedy univerzální překladač, ale tzv. domain-based, což znamená, že bude umět pracovat jen v určité doméně neboli oblasti -- v našem případě jednání Evropského parlametu. Tento tutoriál je inspirovaný [odsud](https://github.com/Hvass-Labs/TensorFlow-Tutorials#).

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import os

from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

C:\Users\katie\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Nejprve si načteme data a (pokud je nemáme stažená, tak si je stáheneme) vyrobíme si také symboly pro označení začátku a konce sekvence.

In [2]:
mark_start = 'ssss '
mark_end = ' eeee'
language_code='cs'

def load_data(english=True, language_code="da", start="", end=""):
    data_dir='data'
    if english:
        filename = "europarl-v7.{0}-en.en".format(language_code)
    else:
        filename = "europarl-v7.{0}-en.{0}".format(language_code)  
    path = os.path.join(data_dir, filename)

    with open(path, encoding="utf-8") as file:
        texts = [start + line.strip() + end for line in file]
    return texts

data_src = load_data(english=False,language_code=language_code)
data_dest = load_data(english=True,language_code=language_code,start=mark_start,end=mark_end)

Když máme nahraná data, tak uděláme jejich preprocessing. K tomu použijeme třídu ```Tokenizer```, která rozseká texty na slova, vytvořit slovník unikátních slov, kterým přiřadíme čísla, a potom každé slovo v datech nahradit jeho číselnou hodnotou. Navíc mohou mít sekvence různé délky, takže je potřeba nastavit všem stejnou, tedy některé zkrátit a jiné dopaddovat. Délku nastavíme tak, aby bylo potřeba zkrátit je asi 5% sekvencí, což je poměrně dobrý kompromis vzhledem k velikosti paměti a zachování dat. 

In [3]:
# Tokenizer k nahrazení slov číselnou hodnotou
num_words = 10000
class TokenizerWrap(Tokenizer):
    def __init__(self, texts, padding, reverse=False, num_words=None):
        Tokenizer.__init__(self, num_words=num_words)

        # Vytvoření slovníku z textu
        self.fit_on_texts(texts)

        # Vytvoření inverzního vyhledávání od celočíselných tokenů ke slovům.
        self.index_to_word = dict(zip(self.word_index.values(),self.word_index.keys()))

        # Převod všech textů na seznam celočíselných tokenů
        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            # Obrácení sekvenece tokenů
            self.tokens = [list(reversed(x)) for x in self.tokens]
            # Příliš dlouhé obrácené sekvence by měly být zkráceny na začátku, což odpovídá konci původních.
            truncating = 'pre'
        else:
            # Příliš dlouhé sekvence by měly být zkráceny na konci.
            truncating = 'post'

        # Zjištění počtu celočíselných tokenů v každé sekvenci
        self.num_tokens = [len(x) for x in self.tokens]

        # Doplnění nebo zkrácení sekvencí na stejnou délku.
        self.max_tokens = np.mean(self.num_tokens) + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)
        self.tokens_padded = pad_sequences(self.tokens, maxlen=self.max_tokens, padding=padding, truncating=truncating)
    
    # Vyhledání slova z celočíselného tokenu
    def token_to_word(self, token):
        word = " " if token == 0 else self.index_to_word[token]
        return word 

    # Převod seznamu celočíselných tokenů na řetězec
    def tokens_to_string(self, tokens):

        # Vytvoření listu slov
        words = [self.index_to_word[token] for token in tokens if token != 0]
        
        # Spojení slov do stringu pomocí mezery
        text = " ".join(words)
        return text

    # Convert a single text-string to tokens with optional reversal and padding
    def text_to_tokens(self, text, reverse=False, padding=False):

        # Převod textu na tokeny
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:
            # Obrácení tokenu
            tokens = np.flip(tokens, axis=1)

            # Příliš dlouhé obrácené sekvence by měly být zkráceny na začátku, což odpovídá konci původních.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated at the end.
            truncating = 'post'
        if padding:
            # Příliš dlouhé sekvence by měly být zkráceny na konci.
            tokens = pad_sequences(tokens,  maxlen=self.max_tokens, padding='pre', truncating=truncating)

        return tokens

tokenizer_src = TokenizerWrap(texts=data_src,padding='pre',reverse=True,num_words=num_words)
tokenizer_dest = TokenizerWrap(texts=data_dest, padding='post', reverse=False,num_words=num_words)
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_end = tokenizer_dest.word_index[mark_end.strip()]

Nyní už jen data preprocessovaná do celočíselných tokenů vhodně předáme encoderu a decoderu, abychom mohli na trénovat naši neuronovou síť.

In [4]:
encoder_input_data = tokens_src
decoder_input_data = tokens_dest[:, :-1]
decoder_output_data = tokens_dest[:, 1:]
print(decoder_input_data.shape)
print(decoder_output_data.shape)

(646605, 52)
(646605, 52)


Pak už můžeme vytvořit neuronovou síť. Začneme tím, že si naprogramujeme encoder, který se bude skládat ze vstupní vrstvy, vrstvy embeddingů, 3 GRU vrstev a výstupu. Následovat bude decoder, který jako vstup bere výstup encoderu a bude se opět skládat ze vstupní vrsty, 3 GRU vrstev a výstupní vrstvy, která bude převádět výstup na one-hot zakódované pole. Ještě si definujeme ztrátovou funkci ```sparse_cross_entropy``` a pak můžeme model přeložit.

In [5]:
# Vytvoření encoderu
encoder_input = Input(shape=(None, ), name='encoder_input')

embedding_size = 128
encoder_embedding = Embedding(input_dim=num_words, output_dim=embedding_size, name='encoder_embedding')

state_size = 512
encoder_gru1 = GRU(state_size, name='encoder_gru1', return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2', return_sequences=True)
encoder_gru3 = GRU(state_size, name='encoder_gru3', return_sequences=False)

# Spojení jednotlivých vrstev
def connect_encoder(): 
    net = encoder_input
    net = encoder_embedding(net)
    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)
    encoder_output = net  
    return encoder_output

encoder_output = connect_encoder()

# Vytvoření decoderu
decoder_initial_state = Input(shape=(state_size,), name='decoder_initial_state')
decoder_input = Input(shape=(None, ), name='decoder_input')
decoder_embedding = Embedding(input_dim=num_words, output_dim=embedding_size, name='decoder_embedding')
decoder_gru1 = GRU(state_size, name='decoder_gru1', return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2', return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3', return_sequences=True)
decoder_dense = Dense(num_words, activation='linear', name='decoder_output')

# Spojení jednotlivých vrstev
def connect_decoder(initial_state):

    net = decoder_input
    net = decoder_embedding(net)
    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)
    decoder_output = decoder_dense(net)
    
    return decoder_output

# Spojení a vytvoření modelu
decoder_output = connect_decoder(initial_state=encoder_output)
model_train = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])
model_encoder = Model(inputs=[encoder_input], outputs=[encoder_output])
decoder_output = connect_decoder(initial_state=decoder_initial_state)
model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs=[decoder_output])

# Ztrátová funkce
def sparse_cross_entropy(y_true, y_pred):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    loss_mean = tf.reduce_mean(loss)
    return loss_mean

# Sestavení modelu
optimizer = RMSprop(lr=1e-3)
decoder_target = tf.placeholder(dtype='int32', shape=(None, None))
model_train.compile(optimizer=optimizer, loss=sparse_cross_entropy, target_tensors=[decoder_target])


Ještě se nám může hodit callback funkce pro zapisování checkpointů během trénování.

In [6]:
# Callback function
path_checkpoint = '21_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint, monitor='val_loss', verbose=1,
                                      save_weights_only=True, save_best_only=True)
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
callback_tensorboard = TensorBoard(log_dir='./21_logs/', histogram_freq=0, write_graph=False) 
callbacks = [callback_early_stopping, callback_checkpoint, callback_tensorboard]

Nyní už se můžeme pustit do samotného trénovaní modelu na našich datech. V průběhu trénování si ukládáme po každé epoše checkpointy, takže po každé budeme mít k dispozici natrénované váhy.

In [ ]:
x_data = { 'encoder_input': encoder_input_data, 'decoder_input': decoder_input_data }
y_data = { 'decoder_output': decoder_output_data }
 
validation_split = 10000 / len(encoder_input_data)

model_train.fit(x=x_data, y=y_data, batch_size=512, epochs=10, validation_split=validation_split, callbacks=callbacks)

Model na CPU trénovat nebudeme, protože to trvá velice dlouho. Trénování jsem ale pustila za vás na GPU a natrénované váhy najdete přiložené k tomuto cvičení. K samotnému překladu testovacích vět si ještě napíšeme funkci ```translate```, která nám větu preprocessuje a předá ji síti a nechá ji přeložit. To uděláme tak, že vstupní text převedeme na sekvenci tokenů integerů a použijeme výstupní vrstvu encoderu, která bude použita jako počáteční stav v GRU dekodéru. Mohli bychom použít i koncový stav encoderu, ale to je skutečně nutné pouze tehdy, pokud encoder a decoder používají LSTM místo GRU, protože LSTM má dva vnitřní stavy. 

Potom dokud nenarazíme na konec sekvence nebo maximální počet tokenů budeme opakovat aktualizaci vstupní sekvence decoderu posledním vzorkovaným tokenem, v první iteraci nastavíme token na startovní. Pak si data zabalíme do slovníku pro přehlednost a bezpečnost,že jsme je zapsali ve správném pořadí a předáme je decoderu, aby nám predikoval překlad. Tokeny potom převeme na slova pomocí slovníku a ty sloučíme pomocí mezer do jednoho stringu a ten si vypíšeme.

In [7]:
# Překlad jednoho stringu textu
def translate(input_text, true_output_text=None):
    # Převod vstupního textu do tokenů integerů
    input_tokens = tokenizer_src.text_to_tokens(text=input_text, reverse=True, padding=True)
    
    # Získání výstupní vrstvy encoderu
    initial_state = model_encoder.predict(input_tokens)

    # Nastavení maximálního počtu tokenů v sekcenci
    max_tokens = tokenizer_dest.max_tokens

    # Prealokace 2D pole pro vstup decoderu 
    shape = (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)

    # První vstupní token musí být 'ssss '.
    token_int = token_start

    # Inicializace prázdného výstupního textu
    output_text = ''

    # Inicializace počtu už zpracovaných tokenů
    count_tokens = 0

    # Dokud není splněno ukočnovací kritérium
    while token_int != token_end and count_tokens < max_tokens:

        # Update vstupní sekvence pro decoder
        decoder_input_data[0, count_tokens] = token_int

        # Zabalení vstupních dat do slovníku 
        x_data = {'decoder_initial_state': initial_state, 'decoder_input': decoder_input_data }

        # Předání dat decoderu a predikce
        decoder_output = model_decoder.predict(x_data)

        # Získání posledního predikovaného tokenu jako one-hot pole
        token_onehot = decoder_output[0, count_tokens, :]
        
        # Převod na pole integerů
        token_int = np.argmax(token_onehot)

        # VYhledání slova odpovídající tokenu
        sampled_word = tokenizer_dest.token_to_word(token_int)

        # Přidání do výstupního textu
        output_text += " " + sampled_word
            
        # Zvýšení counteru počtu tokenů
        count_tokens += 1

    # Sekvence tokenů vrácená decoderem
    output_tokens = decoder_input_data[0]
    
    # Výpisy originální věty a překladu bez symbolů začátku a konce
    print("Input text:")
    print(input_text)
    print()

    print("Translated text:")
    if output_text[0]==' ':
        output_text = output_text[1:]
    if 'ssss' in output_text:
        output_text = output_text[5:]
    if 'eeee' in output_text:
        output_text = output_text[:-5]
    print(output_text)
    print()

    # Volitelná možnost výpisu skutečného překladu
    if true_output_text is not None:
        print("True output text:")
        if 'ssss' in true_output_text:
            true_output_text = true_output_text[5:]
        if 'eeee' in true_output_text:
            true_output_text = true_output_text[:-5]
        print(true_output_text)
        print()

Zkusíme si tedy načíst natrénované váhy modelu a otestovat si ho pomocí překladu nějakých vět. 

In [8]:
# Načtění checkpointu neboli vah do našeho modelu
try:
    model_train.load_weights(path_checkpoint)
    print('Weights loaded')
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

Weights loaded


Nejprve se podíváme na překlad několika náhodně zvolených vět, na kterých jsme model trénovali a zároveň si i porovnáme překlad s původní větou. Všimněme si, že překladač vrací výsledek bez rozlišení velikosti písmen. To proto, že vstupní a výstupní texty jsou před trénováním lowercasované, aby to měl překladač snažší a nemusel rozlišovat ješt i velikosti písmen.

In [9]:
idx = 3
translate(input_text=data_src[idx], true_output_text=data_dest[idx])
print('=====')

Input text:
Texty smluv dodané Radou: viz zápis

Translated text:
texts of agreements forwarded by the council see minutes

True output text:
Texts of agreements forwarded by the Council: see Minutes

=====


Teď zkusíme přeložit i některé náhodné věty, které si sami vymyslíme. Nejprve se pokusíme vymyslet věty, které by mohly sedět do naší domény, tedy by mohly zaznít v jednání Evropského parlamentu.

In [10]:
translate(input_text="Česká republika je členem Evropské Unie", true_output_text='The Czech Republic is a member of the European Union')
print('=====')
translate(input_text="Vláda se rozhodla projednat nová pravidla pro cestování", true_output_text='The government has decided to discuss new rules for travelling')
print('=====')
translate(input_text="Francie tento návrh rozhodně nepodpoří", true_output_text='France will certainly not support this proposal')

Input text:
Česká republika je členem Evropské Unie

Translated text:
the czech republic is a member of the european union

True output text:
The Czech Republic is a member of the European Union

=====
Input text:
Vláda se rozhodla projednat nová pravidla pro cestování

Translated text:
the government has decided to discuss new rules on travel

True output text:
The government has decided to discuss new rules for travelling

=====
Input text:
Francie tento návrh rozhodně nepodpoří

Translated text:
france has certainly not made it

True output text:
France will certainly not support this proposal



Nyní si pro zajímavost zkusíme vymyslet nějaké věty, které určitě z naší domény nejsou, tedy s největší pravděpodobností v žádném jednání Evropského parlamentu nezazněla, i když jeden nikdy neví. :)

In [11]:
translate(input_text="Dneska mě tu to fakt nebaví", true_output_text='I really don\'t like it today')
print('=====')
translate(input_text="Můj pes mi ukradl noviny", true_output_text='My dog stole my newspaper')

Input text:
Dneska mě tu to fakt nebaví

Translated text:
i am surprised that this is a reality

True output text:
I really don't like it today

=====
Input text:
Můj pes mi ukradl noviny

Translated text:
my dear colleagues

True output text:
My dog stole my newspaper

